In [20]:
# %pip install -U langchain-ollama
# %pip install -U ollama

# https://python.langchain.com/docs/integrations/chat/ollama/
# https://python.langchain.com/docs/integrations/vectorstores/faiss_async/
# https://python.langchain.com/docs/how_to/migrate_agent/#in-langgraph-2

import langchain

langchain.__version__

'0.3.12'

In [ ]:
from langchain_core.messages import AIMessage
from langchain.prompts import PromptTemplate
from langchain_ollama import ChatOllama
from langchain_core.runnables import RunnablePassthrough


docs = [
    "This restaurant is great. The food is delicious and the service is excellent.",
    "AIMessage is a great tool for building chatbots.",
    "I love the new iPhone. It has a great camera and the battery life is amazing.",
    "Hello, how are you?",
    "I am doing well. Thank you for asking.",
]
docs_str = ""
for doc in docs:
    docs_str += f"<doc># Docuent {docs.index(doc) + 1}:\n"
    docs_str += f"{doc}</doc>\n"


llm = ChatOllama(model="llama3.1", temperature=0, verbose=True)

# prompt = PromptTemplate(
#     name="Document Curation Chatbot",
#     input_variables=["query", "docs"],
#     template=
prompt = PromptTemplate.from_template(
    """당신은 여러개의 문서를 가지고 분석하여 사용자에게 큐레이션을 할 수 있는 챗봇입니다.
아래의 문서들을 분석하여 사용자의 입력에 제일 적합한 내용 하나만을 선택해 제공해주세요.
다음 세 가지 기준을 고려하여 선택해주세요.
1. Direct relevance of the document to the user's input
2. The quality of the document in terms of informativeness and readability
3. The diversity of the documents selected
각 문서는 `<doc>`와 `</doc>`로 감싸져 있습니다.
사용자의 입력은 `<user>`와 `</user>`로 감싸져 있습니다. 사용자의 입력은 항상 마지막에 위치합니다.
당신의 답변은 선택된 문서를 `<doc>`와 `</doc>`로 감싸져 있어야 합니다. 다른 내용은 없어야 합니다.
---

<user>{query}</user>
{docs}
""",
)

chain = prompt | llm
user_query = "폰 새로 사야하는데 추천좀."
output = chain.invoke({"query": user_query, "docs": docs_str})

print("=== 유저 입력 ===")
print(user_query)
print("=== 챗봇 출력 ===")
print(output.content)

prompt2 = PromptTemplate.from_template(
    """You are given user's input and a list of curated documents for the user.
Analyze the input and the documents and answer the user in a positive and helpful manner.
The user's input is wrapped in <user> and </user> tags. The user's input is always at the end.
The curated documents are wrapped in <doc> and </doc> tags.
---
<user>{query}</user>
{docs}
"""
)
chain2 = prompt2 | llm
output2 = chain2.invoke({"query": user_query, "docs": output.content})

print("=== 유저 입력 ===")
print(user_query)
print("=== 챗봇 출력 ===")
print(output2.content)

=== 유저 입력 ===
폰 새로 사야하는데 추천좀.
=== 챗봇 출력 ===
<doc># Docuent 3:
I love the new iPhone. It has a great camera and the battery life is amazing.</doc>
=== 유저 입력 ===
폰 새로 사야하는데 추천좀.
=== 챗봇 출력 ===
It sounds like you're looking to buy a new phone!

You might want to consider getting an iPhone, based on what I've read about them. They seem to have some really great features, such as a fantastic camera and impressive battery life.

Would you like me to suggest more options or provide more information about iPhones?
